In Draft, tested were a few approaches, however Basilisk and Basilisk reinforecement learning environment were not stable, prone to errors and were difficult to configure. Continuing with a second approach: implementing a custom spacecraft reincforecemtn learning environemnt.

Using Gymnasium docs: https://gymnasium.farama.org/introduction/create_custom_env/

#### Custom Gym Environment Arhcitecture
This section discusses the environment as a whole and identifies overall considerations, goals and design.

- compute environment state into observation;
- `reset()` function to initiate a new episode for the environment;
- `step()` to compute the new state of the environment;


When will a reward be given? 
- spacecraft's target destination;
- efficiency or precision;

When will termination occure?
- goal reached;
- loss of fuel;
- collision with another object;
- error in path resulting to the final destination being impossible to reach;

Agent Goals:
- Landing: Soft-landing on a planetary surface;
- Interplanetary Transfer: Navigating from one celestial body to another;
- Orbit Change: Transitioning between different orbits efficiently;

**Observation space:**
- perceiving the environment;
- include enough details and information for the agent;
- consider: framce of reference, coordinate system (Cartesian, Keplerian), sensor errors (noise);

State variables:

| Variable      | Description   |
| ------------- | ------------- |
| $ x, y, z $ | Position in space (Cartesian) or orbital elements |
| $ v_x, v_y, v_z $ | Velocity vector |
| $ \theta, \phi, \psi $ | Orientation (Euler angles) |
| $ w_x, w_y, w_z $ | Angular velocity |
| $ f $ | Remaining fuel |
| $ d_{target} $ | Distance to target |

**Action space:**
- thruster commands;
- consider: thruster limitation, fuel efficiency (more thrust = more fuel consumption), control delays;

Action types:

| Action Type      | Example   |
| ------------- | ------------- |
| Continuous thrust | [Thrust in X, Y, Z] |
| Discrete thrusters | ON/OFF for RCS jets |
| Rotational control | 	[Torque in Roll, Pitch, Yaw]|


**Physics:**  
Physics considerations and implementation plus formulas and general physics laws.  

- Newtonian Dynamics:  
For simpler free space and no gravity motion.  
$ F = ma $ to update velocity and position (Newton's Second Law of Motion states that when a force acts on an object, it will cause the object to accelerate. The larger the object's mass, the greater the force will need to be to cause it to accelerate. This Law may be written as force = mass x acceleration).


- Otbital Mechanics:  
Consider orbital mechanics and forces from celestial bodies.  
Every object in the universe attracts every other body in the universe with a force directed along the line of centers of the two objects
that is proportional to the product of their masses and inversely proportional to the square of the distance between them,
$$ F=\frac{G_{m_1 m2}}{d^2} $$
In this relation, $G$ is the Newton gravitational constant, $m_1$ and $m_2$ are the
masses of the primary and secondary bodies, and $d$ is the distance between them.  
Kepler’s Equation: it relates the Mean Anomaly ($𝑀$) to the Eccentric Anomaly ($𝐸$) for an orbit with eccentricity $𝑒$:
$$ M = E - e \sin E $$
where:
    - $M$ = Mean Anomaly ($M = n(t - t_0)$), where $n = \frac{2_\pi}{T}$ is the mean motion, $T$ is the orbital period, and $t_0$ is a reference epoch.
    - E = Eccentric Anomaly.
    - e = Orbital eccentricity.  
    Once $E$ is found, the True Anomaly $ν$ (the angle between the position vector and the periapsis) can be determined using:
$$ \tan \frac{v}{2} = \sqrt{\frac{1+e}{1-e}} \tan \frac{E}{2}$$
The position in the orbital plane is given by:
$$ r = \frac{a(1 - e^2))}{1 + e \cos v}$$
where:
    - $r$ = radial distance from the focus (central body).  
    - $a$ = semi-major axis.  
  The Cartesian coordinates in the orbital plane (before transformation to an inertial frame) are:
$$ x = r \cos v$$ $$ y = r \sin v $$
Implement with: https://www.astropy.org/ and https://docs.poliastro.space/en/stable/index.


- Attitude Control:  
Rotational dynamics for docking, planetery entry, or pointing. Usega of *quaternions* for rotation.
Euler's theorem: The Orientation of a body is uniquely specified by a vector giving the direction of a body axis and a scalar specifying a rotation angle about the axis. 

- Fuel Consumption Model:  
Thrust burns should reduce remaining fuel. Thruster inefficiencies should be considered. Usage of Tsiolkovsky’s rocket equation:
$$ \varDelta v = v_e ln (\frac{m_i0}{m_f}) $$
where:
    - $\varDelta v$ is the change in velocity (delta-v);
    - $ v_e $ is the exhaust velocity;
    - $ m_0 $ is the initial mass of the rocket;
    - $ m_f $ is the final mass of the rocket after the propellant is expended;

#### Reward Function
Reward function should be shaped well and not only the end goal. The primary goal of the agent should be to reach the target, minimize fuel consumption, maintain stable motion, avoid oscillations, collisions and unsafe conditions. The reward function should balance all these goals.

| Reward Type      | Description  |
| ------------- | ------------- |
| Goal proximity | Reward for getting closer to the target |
| Fuel efficiency | Penalize excessive fuel use |
| Soft landing | Bonus for smooth landings (low velocity) |
| Docking precision | Bonus for aligning position & velocity |
| Stability | Penalize excessive rotations |

*Phase 1*: Reward based on moving towards the target.  
*Phase 2*: Reward for slowing down near the target.  
*Phase 3*: Reward only for successful docking.  

A combiation of dense reward and spacrse reward can be applyed:
- Dense rewards can speed up learning and will provide continuous feedback.
- Sparse rewards will the given on task completion, but can be more challening for this scenario.

**Reward function design:**
1. Proximity: a proximity reward is provided to ensure that the spacecraft moves towards the goal. A **composite reward function** is used to combine multiple rewards.  
*Composite rewards combine multiple reward signals into a single function. This is useful in complex tasks where multiple objectives need to be balanced.*  
The distance based-reward will be calculated with the negative Euclidean distance:
$$ r_{distance} = - \lVert x - x_{target} \rVert $$
where $x$ is the current position and $x_{target}$ is the target position.
> ```python
> reward_distance = -np.linalg.norm(self.state[:3] - self.target_position)

2. Efficiency: minimizes excessive thrust burns and can penalize thurs usage.  
*Fuel penalty* where $a$ is the applied thrust vector:
$$ r_{fuel} = - \lVert a \rVert $$
> ```python
> reward_fuel = -np.linalg.norm(action) * 0.01
> 
For fuel *conservation*:  
$$ r_{fuel} = \frac{f_{remaining}}{f_{max}} $$
> ```python
> reward_fuel = self.state[-1] / self.max_fuel
>
3. Velocity: the agent needs to match the correct velocity and not overshoot. The velocity for precise docking has to be correct not to overshoot.
$$ r_{velocity} = - \lVert v - v{target} \rVert $$
> ```python
> reward_velocity = -np.linalg.norm(self.state[3:6] - self.target_velocity)
>
4. Orientation: in order to dock a spacecraft needs to align correctly and penalizing misalignment:
$$ r_{orientation} = -angle(q, q_{target}) $$
where $q$ is the quaternion representing orientation.
5. Landing: soft landing and encouraging stability. High velocity should be controlled during ladning anv penalized if the speed is to high:
$$ r_{ladning} = -v^2_z $$
6. Terminal rewards: in order to achiave correct behaviour, usage of large terminal rewards will be applyed for successfull missions. There will be a large penalty for failiures (for example crashes). Avoiding agent guessing and maximizing learning efficiency.

A weighted sum of all the components of the reward function:
$$ R = w1r_{distance} + w2r_{fuel} + w3r_{velocity} + w4r_{orientation} + w5r_{landing} + r_{goal} + r_{crash} $$

```python
reward = (
    -np.linalg.norm(self.state[:3] - self.target_position) * 1.0  # Distance penalty
    -np.linalg.norm(action) * 0.01  # Fuel efficiency penalty
    -np.linalg.norm(self.state[3:6] - self.target_velocity) * 0.5  # Velocity matching
    + orientation_penalty(self.state[6:10], self.target_orientation) * 0.3  # Orientation alignment
    + (100 if docking_success else 0)  # Goal reward
    + (-100 if collision_detected else 0)  # Collision penalty
)
```
Avoid and consider:
- avoid unintended ways to for the agent to maximize rewards;
- sparce rewards can lead to less exploration of useful actions;
- avoid over-penalizing fuel consumption since the agent might not move;
- scalling reward terms in similar magnitude to achieve equall learning;

#### Training and Evaluation
Algorithms: Proximal Policy Optimization (PRO), Soft Actor-Critic (SAC), Twin Delayed DDPG (TD3)  
Hyperparameter tuning, learning rate, batch size, exploration strategies  

Evaluation Metrics:
1. Final Orbit Accuracy – Does the spacecraft reach the target orbit within a small margin?
2. Fuel Efficiency – Does the agent minimize fuel usage?
3. Number of Maneuvers – Does the agent execute a minimal number of burns?
4. Success Rate – Percentage of successful orbital insertions across test episodes.
5. Time Taken – How many simulation steps are needed to reach the target?



#### Implementation

Inherit base `Gym` environment to define custom environment.  
Set a base space environment and reuse at with wrappers for other environment setups like docking, orbit, etc.

In [2]:
# Imports
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from gymnasium.wrappers import RescaleAction
from gymnasium.envs import register

import ray
from ray import tune
from ray.tune.registry import register_env
from ray.air import RunConfig
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.ppo import PPO
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.algorithms.algorithm_config import AlgorithmConfig

In [2]:
# Init navigation env
class SpaceCraftNavigationEnv(gym.Env):
    '''A basic custom spacecraft environment. Used as a base wrapper.'''
    def __init__(self, config=None):
        super().__init__()
                
        # Define observation space (position, velocity, fuel)
        self.observation_space = spaces.Box(
            low=np.array([-1.0, -1.0, -1.0, -1.0, 0.0]),  # Min values (position, velocity, fuel)
            high=np.array([1.0, 1.0, 1.0, 1.0, 1.0]),     # Max values
            dtype=np.float32
        )

        # Define action space (thrust x, thrust y)
        self.action_space = spaces.Box(
            low=np.array([-1.0, -1.0]), 
            high=np.array([1.0, 1.0]), 
            dtype=np.float32
        )

        self.state = None  # Placeholder for spacecraft state
        self.reset()

    def reset(self, seed=None, options=None):
        # Reset state: Random position, zero velocity, full fuel
        self.state = np.array([np.random.uniform(-1, 1), np.random.uniform(-1, 1), 0.0, 0.0, 1.0], dtype=np.float32)
        return self.state, {}

    def step(self, action):
        x, y, vx, vy, fuel = self.state
        thrust_x, thrust_y = action

        # Update velocity based on action
        vx += thrust_x * 0.01
        vy += thrust_y * 0.01

        # Update position
        x += vx
        y += vy

        # Consume fuel
        fuel = max(0, fuel - np.linalg.norm(action) * 0.01)

        # Define reward (goal: reach [0,0] with fuel efficiency)
        reward = -np.linalg.norm([x, y]) + fuel

        # Check termination (reached goal or out of fuel)
        done = (np.linalg.norm([x, y]) < 0.1) or (fuel <= 0)

        self.state = np.array([x, y, vx, vy, fuel], dtype=np.float32)
        return self.state, reward, done, False, {}

    def render(self):
        print(f"Position: {self.state[:2]}, Velocity: {self.state[2:4]}, Fuel: {self.state[4]}")

In [20]:
# Register the environment with RLlib
register_env("Spacecraft-v0", lambda config: SpaceCraftNavigationEnv(config))
register(
    id="Spacecraft-v0",
    entry_point="__main__:SpaceCraftNavigationEnv",
    kwargs={"config": None}
)

C:\Users\zhech\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\envs\registration.py:642: UserWarning: WARN: Overriding environment Spacecraft-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")


In [5]:
# Train an agent with RLlib PPO
# Initialize Ray
ray.init(ignore_reinit_error=True)
# Configure PPO training
config = (
    PPOConfig()
    .environment("Spacecraft-v0")
    .env_runners(num_env_runners=2) # parallel rollouts
    .training(
        gamma=0.99, #discount factor ([0-1] closer to 1 makes the agent prioritize long-term rewards)
        lr=1e-3, #learning rate
        train_batch_size=4000
    )
    .framework("torch") 
)

# Train using Tune
tuner = tune.Tuner(
    "PPO",
    param_space=config.to_dict(),
    run_config=RunConfig(stop={"training_iteration": 100}), # train for 100 iterations
)
results = tuner.fit()

ray.shutdown()

2025-02-03 22:01:41,841	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-02-03 22:01:41,844	WARNING algorithm_config.py:4702 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-02-03 22:27:26,909	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to 'C:/Users/zhech/ray_results/PPO_2025-02-03_22-01-41' in 0.01

In [6]:
# View results
results.get_best_result()

Result(
  metrics={'timers': {'training_iteration': 15.355961166226495, 'restore_workers': 3.3750257361638816e-05, 'training_step': 15.355455823670509, 'env_runner_sampling_timer': 2.3035685787032194, 'learner_update_timer': 13.041798274708952, 'synch_weights': 0.008705616528840898, 'synch_env_connectors': 0.010918608491008928}, 'env_runners': {'timers': {'connectors': {'TensorToNumpy': np.float64(8.895721404496892e-05), 'RemoveSingleTsTimeRankFromBatch': np.float64(2.430166262614601e-06), 'AddObservationsFromEpisodesToBatch': np.float64(1.2009260273826356e-05), 'BatchIndividualItems': np.float64(2.8675487066060734e-05), 'AddStatesFromEpisodesToBatch': np.float64(1.0247948988452359e-05), 'NumpyToTensor': np.float64(4.7400056533005335e-05), 'UnBatchToIndividualItems': np.float64(3.178074787617437e-05), 'NormalizeAndClipActions': np.float64(8.404728050171263e-05), 'ListifyDataForVectorEnv': np.float64(4.738487275675073e-05), 'GetActions': np.float64(0.00032133654504573346)}}, 'episode_du

In [3]:
# Load the trained policy
algo = PPO.from_checkpoint(results.get_best_result().checkpoint.path)

# Test the trained agent
env = SpaceCraftNavigationEnv()
obs, _ = env.reset()

done = False
while not done:
    # action = algo.compute_single_action(obs)
    action = algo.compute_single_action(obs, explore=False)
    obs, reward, done, _, _ = env.step(action)
    env.render()

NameError: name 'results' is not defined

Evaluation error BUG: https://github.com/ray-project/ray/issues/44475

In [ ]:
# Entry point evaluation
algo = PPO.from_checkpoint("C:/Users/zhech/ray_results/PPO_2025-02-02_19-34-57/PPO_Spacecraft-v0_05501_00000_0_2025-02-02_19-34-57/checkpoint_000000")
env = gym.make("Spacecraft-v0")
obs, info = env.reset()
for _ in range(1000):
    action = algo.compute_single_action(obs)
    obs, reward, done, truncated, info = env.step(action)
    if done or truncated:
        obs, info = env.reset()

#### Experiments
Perform multiple experiments and tests in separete notebooks.

![Figure_approach](Figures/Approach_01.png)

Agent = Algorithm: **PPO, SAC** for our tasks.
How can these algorithms perform on our environments? Can the algorithm's hyperparameters be tuned to learn and grow it's rewards? Or should the environment be tuned, so that the algorithm is more effective?  
How well can these algorithms perform on our environments? What do we measure?  

How many time steps will each trial or run contain? If a terminal state is not reached after n steps, will we artificially terminate the episode? If the task is episodic, how many episodes should we run?  

